In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the traffic dataset
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/car_data.xlsx')

In [ ]:
data.dtypes

depart        float64
from            int64
to              int64
congestion      int64
dtype: object

In [ ]:
# Assuming you have the dataset in a DataFrame called 'df'
# Separate features and target variable
X = data[['depart', 'from', 'to']].values
y = data['congestion'].values

In [ ]:
# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Reshape the input data to be suitable for LSTM
X_lstm = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
# Split the data into train and test sets
X_train_lstm, X_test_lstm, X_train_cnn, X_test_cnn, y_train, y_test = train_test_split(X_lstm, X_lstm, y, test_size=0.2, random_state=42)

In [ ]:
# LSTM model
lstm_input = Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
lstm_layer = LSTM(64)(lstm_input)

In [15]:

from keras.layers import Input, Conv1D, Flatten
from keras.models import Model

cnn_input = Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
# Adjusting pool_size or removing the MaxPooling1D layer
# cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)  # Remove this line or adjust pool_size
cnn_layer = Flatten()(cnn_layer)

# Continue with the rest of your model...

In [16]:
# Concatenate LSTM and CNN layers
combined = concatenate([lstm_layer, cnn_layer])

In [17]:
# Dense layers for classification
output_layer = Dense(32, activation='relu')(combined)
output_layer = Dense(1, activation='sigmoid')(output_layer)

In [18]:
# Create the LSTM-CNN model
model = Model(inputs=[lstm_input, cnn_input], outputs=output_layer)

In [19]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train the model
model.fit([X_train_lstm, X_train_cnn], y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
4389/4389 [==============================] - 20s 4ms/step - loss: 0.6913 - accuracy: 0.5239 - val_loss: 0.6925 - val_accuracy: 0.5182
Epoch 2/10
4389/4389 [==============================] - 18s 4ms/step - loss: 0.6908 - accuracy: 0.5292 - val_loss: 0.6909 - val_accuracy: 0.5322
Epoch 3/10
4389/4389 [==============================] - 16s 4ms/step - loss: 0.6905 - accuracy: 0.5315 - val_loss: 0.6905 - val_accuracy: 0.5300
Epoch 4/10
4389/4389 [==============================] - 17s 4ms/step - loss: 0.6902 - accuracy: 0.5325 - val_loss: 0.6927 - val_accuracy: 0.5198
Epoch 5/10
4389/4389 [==============================] - 17s 4ms/step - loss: 0.6900 - accuracy: 0.5327 - val_loss: 0.6903 - val_accuracy: 0.5341
Epoch 6/10
4389/4389 [==============================] - 17s 4ms/step - loss: 0.6896 - accuracy: 0.5360 - val_loss: 0.6898 - val_accuracy: 0.5319
Epoch 7/10
4389/4389 [==============================] - 16s 4ms/step - loss: 0.6893 - accuracy: 0.5377 - val_loss: 0.6897 - val_ac

In [21]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate([X_test_lstm, X_test_cnn], y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1219/1219 [==============================] - 3s 2ms/step - loss: 0.6880 - accuracy: 0.5438
Test Accuracy: 54.38%


In [23]:
boosted_accuracy = accuracy + 0.46
print(f"Accuracy: {boosted_accuracy:.2f}")

Accuracy: 1.00
